In [1]:
import pandas as pd 

In [ ]:
df = pd.read_excel("apex_bio.xlsx", sheet_name="Chemical Data") 

In [4]:
import pubchempy as pcp

/Users/jadedandois/Desktop/Cours UniFR/3ème Année /SP/Travail de Bachelor /apexbio_data_cleaning/.venv/lib/python3.12/site-packages/pubchempy.py:563: SyntaxWarning: "is not" with 'int' literal. Did you mean "!="?
  if self.charge is not 0:


In [44]:
cas_numbers = df["CAS Number"].to_list()

In [27]:
cas_numbers

['4474-91-3',
 '1953-04-4',
 '2068-78-2',
 '21535-47-7',
 '25316-40-9',
 '102518-79-6',
 '446-72-0',
 '476-66-4',
 '56180-94-0',
 '59937-28-9',
 '56-75-7',
 '31282-04-9',
 '25389-94-0',
 '64-75-5',
 '6151-25-3',
 '989-51-5',
 '34157-83-0',
 '487-52-5',
 '73-31-4',
 '1968-05-4',
 '104987-12-4',
 '477-47-4',
 '472-15-1',
 '33570-04-6',
 '72962-43-7',
 '404-86-4',
 '64-86-8',
 '458-37-7',
 '518-82-1',
 '357-70-0',
 '2752-65-0',
 '525-79-1',
 '1180-71-8',
 '519-02-8',
 '500-38-9',
 '60-82-2',
 '38748-32-2',
 '38647-11-9',
 '143-67-9',
 '24390-14-5',
 '30562-34-6',
 '1069-66-5',
 '501-36-0',
 '362-07-2',
 '4759-48-2',
 '75330-75-5',
 '33069-62-4',
 '114977-28-5',
 '32981-86-5',
 '50-76-0',
 '50-07-7',
 '20069-09-4',
 '18883-66-4',
 '220620-09-7',
 '7681-93-8',
 '79217-60-0',
 '53123-88-9',
 '550-24-3',
 '66-81-9',
 '5908-99-6',
 '5593-20-4',
 '50-99-7',
 '1961-12-1',
 '50-28-2',
 '60142-96-3',
 '59-92-7',
 '6080-58-6',
 '434-13-9',
 '31430-18-9',
 '57-83-0',
 '79902-63-9',
 '1953-2-2',
 '30

In [ ]:
liste_cas_numbers_erreurs = []
liste_cas_numbers_ok = []

for x in cas_numbers:
    try : 
        liste_cas_numbers_ok.append(pcp.get_compounds(x, "name", as_dataframe=True))
    except KeyError:
        liste_cas_numbers_erreurs.append(x)



In [36]:
liste_cas_numbers_erreurs

['1961-12-1',
 '1953-2-2',
 '1405-87-4',
 '1264-72-8',
 '5053-8-7',
 '1967-3-8',
 '1950-4-4',
 '5451-9-2',
 '1953-3-2',
 '5875-6-9',
 '76135-82-5',
 '102040-03-9',
 '65995-63-3',
 '32451-88-0',
 '1958-8-2',
 '6873-9-2',
 '6859-1-4',
 '1957-10-3',
 '67656-30-8,4439-81-2,9685-09-7',
 '1994-7-5']

In [38]:
len(liste_cas_numbers_erreurs)

20

In [45]:
x

'84104-71-2'

In [ ]:
donnees = pcp.get_compounds(x, "name", as_dataframe=True)

In [ ]:
liste_cas_numbers = []

for x in cas_numbers:
    try : 
       pcp.get_compounds(x, "name", as_dataframe=True)
    except KeyError:
        continue
    liste_cas_numbers.append(donnees)


In [42]:
liste_cas_numbers

['4474-91-3',
 '1953-04-4',
 '2068-78-2',
 '21535-47-7',
 '25316-40-9',
 '102518-79-6',
 '446-72-0',
 '476-66-4',
 '56180-94-0',
 '59937-28-9',
 '56-75-7',
 '31282-04-9',
 '25389-94-0',
 '64-75-5',
 '6151-25-3',
 '989-51-5',
 '34157-83-0',
 '487-52-5',
 '73-31-4',
 '1968-05-4',
 '104987-12-4',
 '477-47-4',
 '472-15-1',
 '33570-04-6',
 '72962-43-7',
 '404-86-4',
 '64-86-8',
 '458-37-7',
 '518-82-1',
 '357-70-0',
 '2752-65-0',
 '525-79-1',
 '1180-71-8',
 '519-02-8',
 '500-38-9',
 '60-82-2',
 '38748-32-2',
 '38647-11-9',
 '143-67-9',
 '24390-14-5',
 '30562-34-6',
 '1069-66-5',
 '501-36-0',
 '362-07-2',
 '4759-48-2',
 '75330-75-5',
 '33069-62-4',
 '114977-28-5',
 '32981-86-5',
 '50-76-0',
 '50-07-7',
 '20069-09-4',
 '18883-66-4',
 '220620-09-7',
 '7681-93-8',
 '79217-60-0',
 '53123-88-9',
 '550-24-3',
 '66-81-9',
 '5908-99-6',
 '5593-20-4',
 '50-99-7',
 '50-28-2',
 '60142-96-3',
 '59-92-7',
 '6080-58-6',
 '434-13-9',
 '31430-18-9',
 '57-83-0',
 '79902-63-9',
 '302-79-4',
 '50-14-6',
 '69-7

In [43]:
len(liste_cas_numbers)

530

In [11]:
cas_numbers

['4474-91-3',
 '1953-04-4',
 '2068-78-2',
 '21535-47-7',
 '25316-40-9',
 '102518-79-6',
 '446-72-0',
 '476-66-4',
 '56180-94-0',
 '59937-28-9',
 '56-75-7',
 '31282-04-9',
 '25389-94-0',
 '64-75-5',
 '6151-25-3',
 '989-51-5',
 '34157-83-0',
 '487-52-5',
 '73-31-4',
 '1968-05-4',
 '104987-12-4',
 '477-47-4',
 '472-15-1',
 '33570-04-6',
 '72962-43-7',
 '404-86-4',
 '64-86-8',
 '458-37-7',
 '518-82-1',
 '357-70-0',
 '2752-65-0',
 '525-79-1',
 '1180-71-8',
 '519-02-8',
 '500-38-9',
 '60-82-2',
 '38748-32-2',
 '38647-11-9',
 '143-67-9',
 '24390-14-5',
 '30562-34-6',
 '1069-66-5',
 '501-36-0',
 '362-07-2',
 '4759-48-2',
 '75330-75-5',
 '33069-62-4',
 '114977-28-5',
 '32981-86-5',
 '50-76-0',
 '50-07-7',
 '20069-09-4',
 '18883-66-4',
 '220620-09-7',
 '7681-93-8',
 '79217-60-0',
 '53123-88-9',
 '550-24-3',
 '66-81-9',
 '5908-99-6',
 '5593-20-4',
 '50-99-7',
 '1961-12-1',
 '50-28-2',
 '60142-96-3',
 '59-92-7',
 '6080-58-6',
 '434-13-9',
 '31430-18-9',
 '57-83-0',
 '79902-63-9',
 '1953-2-2',
 '30